In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [26]:
# Importar Datasets
users_original = pd.read_pickle('users.pkl')
users_featnue = pd.read_csv('features_generadas.csv')

In [27]:
# Quedarnos solo con las columnas utilizables
users_original = users_original[['Cat','bot','default_profile', 'description', 'geo_enabled', 'lang', 'listed_count', 'location', 'name', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_banner_url', 'profile_image_url', 'profile_image_url_https', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'screen_name', 'url','favs_tiempo','followers_tiempo', 'statuses_tiempo','friends_tiempo']]
users_featnue.drop(['user_id.1','tweetsCount', 'semanas'], axis=1,inplace=True)
# Rellenar valores vacíos
users_featnue = users_featnue.fillna(0)

In [28]:
# Generar ejes X de features y Normalizarlo
X_o = users_original.drop(columns=['Cat','bot'])[users_original.Cat!='SocialSpambot1']
atributos_o = X_o.columns
X_o = preprocessing.scale(X_o, axis = 0)

X_n = users_featnue.drop(columns=['Cat','bot','user_id'])[users_featnue.Cat!='SocialSpambot1']
atributos_n = X_n.columns
X_n = preprocessing.scale(X_n, axis = 0)

X_ot = users_original.drop(columns=['Cat','bot'])[users_original.Cat=='SocialSpambot1']
atributos_o = X_ot.columns
X_ot = preprocessing.scale(X_ot, axis = 0)

X_nt = users_featnue.drop(columns=['Cat','bot','user_id'])[users_featnue.Cat=='SocialSpambot1']
atributos_n = X_nt.columns
X_nt = preprocessing.scale(X_nt, axis = 0)

In [81]:
# Generar ejes Y de targets y encodear las etiquetas
le = preprocessing.LabelEncoder()
y_o =  le.fit_transform(users_original['bot'][users_original.Cat!='SocialSpambot1'])
y_n =  le.fit_transform(users_featnue['bot'][users_featnue.Cat!='SocialSpambot1'])
y_ot =  le.fit_transform(users_original['bot'][users_original.Cat=='SocialSpambot1'])
y_nt =  le.fit_transform(users_featnue['bot'][users_featnue.Cat=='SocialSpambot1'])
y_ot = np.where(y_ot==0, 1, 0)
y_nt = np.where(y_nt==0, 1, 0)

In [37]:
# Separar entre train y test
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_o,y_o)
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X_n,y_n)

In [98]:
# Crear loop para probar n neighbors 
scores_para_df = []
for i in range(20,100,1):  # de 1 a 10 Kn con steps de 1
    # En cada iteración instanciamos el modelo con un hiperparámetro distinto
    model = KNeighborsClassifier(n_neighbors=i)
    # cross_val_scores nos devuelve un array de 5 resultados, uno por cada partición que hizo automáticamente CV  
    cv_scores = cross_val_score(model, X_n, y_n, cv=5)
    # Para cada valor de n_neighbours, creo un diccionario con el valor de n_neighbours y la media y el desvío de los scores.
    dict_row_score = {'score_medio':np.mean(cv_scores),'score_std':np.std(cv_scores),'n_neighbours':i}
    # Guardo cada uno en la lista de diccionarios
    scores_para_df.append(dict_row_score)

In [97]:
# Modelo Original
model_o = KNeighborsClassifier(n_neighbors=45)
model_o.fit(X_train_o,y_train_o)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=45, p=2,
           weights='uniform')

In [100]:
#Modelo Features Nuevas
model_n = KNeighborsClassifier(n_neighbors=92)
model_n.fit(X_train_n,y_train_n)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=92, p=2,
           weights='uniform')

In [102]:
# Predecir valores de test
y_pred_o = model_o.predict(X_test_o)
y_pred_n = model_n.predict(X_test_n)
y_pred_ot = model_o.predict(X_ot)
y_pred_nt = model_n.predict(X_nt)

In [104]:
# Calculamos el accuracy general del modelo
from sklearn.metrics import accuracy_score
print("Modelo Original en Test: ",accuracy_score(y_test_o, y_pred_o))
print("Modelo Nuevas Features en Test: ",accuracy_score(y_test_n, y_pred_n))
print("Modelo orig sin SSB1 en Test: ",accuracy_score(y_ot, y_pred_ot))
print("Modelo nuevo sin SSB1 en Test: ",accuracy_score(y_nt, y_pred_nt))

Modelo Original en Test:  0.9808153477218226
Modelo Nuevas Features en Test:  0.9624300559552358
Modelo orig sin SSB1 en Test:  0.9303733602421796
Modelo nuevo sin SSB1 en Test:  0.9505549949545913


In [107]:
# Matriz de confusión con Modelo con Features Nuevas
confusion_matrix(y_test_o, y_pred_o)

array([[272,  13],
       [ 11, 955]])

In [108]:
# Matriz de confusión con Modelo con Features Nuevas
confusion_matrix(y_test_n, y_pred_n)

array([[257,  15],
       [ 32, 947]])

In [110]:
# Matriz de confusión con Modelo con Features Nuevas
confusion_matrix(y_ot, y_pred_ot)

array([[  0,   0],
       [ 69, 922]])

In [113]:
# Matriz de confusión con Modelo con Features Nuevas
confusion_matrix(y_nt, y_pred_nt)

array([[  0,   0],
       [ 49, 942]])

In [115]:
# Scores con Modelo con Features Nuevas
print(classification_report(y_nt, y_pred_nt))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       1.00      0.95      0.97       991

avg / total       1.00      0.95      0.97       991



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [116]:
# Scores con Modelo con Features Nuevas
print(classification_report(y_test_n, y_pred_n))

             precision    recall  f1-score   support

          0       0.89      0.94      0.92       272
          1       0.98      0.97      0.98       979

avg / total       0.96      0.96      0.96      1251



In [118]:
predicted_cols = ['pred_'+str(c) for c in le.classes_]

In [130]:
confusion_matrix(y_test_n, y_pred_n)

array([[257,  15],
       [ 32, 947]])

In [135]:
conf = confusion_matrix(y_nt, y_pred_nt)
predicted_cols = ['pred_'+str(c) for c in le.classes_]
pd.DataFrame(conf, index= ['real_Human','real_Bot'],columns = ['pred_Human','pred_Bot'])

,pred_Human,pred_Bot
real_Human,0,0
real_Bot,49,942


In [137]:
conf = confusion_matrix(y_test_n, y_pred_n)
predicted_cols = ['pred_'+str(c) for c in le.classes_]
pd.DataFrame(conf, index= ['real_Human','real_Bot'],columns = ['pred_Human','pred_Bot'])

,pred_Human,pred_Bot
real_Human,257,15
real_Bot,32,947
